# Neural network 
1. Ustawienie losowych wag i biasów.
2. Przepuszenie otrzymanych wyników przez funkcje aktywacji aby dodać nielinowość.
3. Obliczenie błędu przewidywań wyników.
4. Uruchomienie propagacji wstecznej aby poprawić skuteczność przewidywanych wyników.
5. Trening modelu.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
def init_weight_bias():
    weights = []
    biases = []
    
    for _ in range(3):
        weights.append(np.random.rand(2, 2) - 0.5)
        biases.append(np.random.rand(1, 2) - 0.5)

    return weights, biases

Wyczytałem, że dobrze jest odjąć 0.5 od losowo wygenerowanej liczby bo stwarza to zakrec który zawiera liczby dodatnie jak i ujemne oraz wartości unikalne które sprzyjają lepszemu uczeniu się modelowi i nie powodują uczenia się w ten sam sposób na tych samych wagach i odchyleniach. Jest to optymalizacja problemu symmetry breaking. Odjęcie od bias wartości 0.5 pomaga w uniknięciu stronniczości w uczeniu maszynowym, jest to podobno standardowa inicjalizacja.

In [3]:
weights, biases = init_weight_bias()

print("Losowe wagi: ", weights)
print("Losowe biasy: ", biases)

Losowe wagi:  [array([[-0.21407272, -0.02111836],
       [-0.27081978, -0.43158991]]), array([[-0.24565067,  0.09779675],
       [ 0.27765294, -0.04461861]]), array([[-0.29790024,  0.14269741],
       [-0.38599311,  0.24999561]])]
Losowe biasy:  [array([[-0.38458626,  0.48449278]]), array([[-0.02397501,  0.41607745]]), array([[-0.401238  , -0.46469339]])]


![alt text](images/warstwa.png "Layer")

In [4]:
# Abstrakcyjna wartwa bazowa
def layer(input_X, output_Y):
    input_X = None
    output_Y = None

    return input_X, output_Y

![alt text](images/sigmoid.png "funkcja aktywacji")

Zdecydowałem się na funkcje aktywacji sigmoid ponieważ wyczytałem, że jest idealna do klasyfkikacji binarnej 0,1 

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

![alt text](images/softmax.png "softmax")

In [6]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

Miałem nadzieję, że funkcja softmax dedykowana do używania na wyjściowej warstwie poprawi wyniki mojego modelu.

![alt text](images/propagacja_do_przodu.png "Propagacja do przodu")

![alt text](images/suma_wazona.png "Suma wazona")

In [7]:
def forward_propagation(x, weights, biases):
    # Pierwszy layer
    z1 = np.dot(x, weights[0]) + biases[0]
    result_activation_function = sigmoid(z1)
    # Drugi layer
    z2 = np.dot(result_activation_function, weights[1]) + biases[1]
    result_activation_function2 = sigmoid(z2)

    # Output layer
    z3 = np.dot(result_activation_function2, weights[2]) + biases[2]
    # output= sigmoid(z3)
    # Użyje jednak funkcji softmax na końcu sieci
    output = softmax(z3)


    return result_activation_function, result_activation_function2, output

Doszlismy do końca sieci więc należy policzyć straty aby poprawić wyniki przewidywań. 

In [8]:
def mse_loss(y_true, output_predict):
    y_true = y_true.reshape(-1, 1) # Reshape danych do tablic 2D
    output_predict = output_predict.reshape(-1, 1) # Reshape danych do tablic 2D
    output_error = mean_squared_error(y_true, output_predict)
    return output_error

Zdecydowałem się zastasować średni błęd kwadratowy ponieważ jest to dość popularna metoda i używałem tego wcześniej do sprawdzenia straty przy przewidywań modelu Linii Regresyjnej.

In [9]:
# Cost function: Binary cross-entropy loss
def compute_cost(y, y_hat):
    m = y.shape[0]                            # Number of samples
    y_hat = np.clip(y_hat, 1e-10, 1 - 1e-10)  # Avoid log(0) errors by clipping predictions

    # Vectorized implementation of binary cross-entropy loss
    cost = (-1 / m) * np.sum((y * np.log(y_hat)) + (1 - y) * np.log(1 - y_hat))
    return cost

![alt text](images/backpropagation.png "Backpropagation")

In [10]:
# def backward_propagation(x, output_error, weights, biases, learning_rate):

#     weights = np.array(weights)
#     biases = np.array(biases)

#     input_error = np.dot(output_error, weights.T)
#     weights_error = np.dot(x.T, output_error)

#     print(weights.shape)
#     print(weights_error.shape)
#     print(output_error.shape)

#     # Aktualizacja wag i baisow
#     for i in range(len(weights) - 1):
#         weights -= learning_rate * weights_error
#         biases -= learning_rate * output_error 

#     return weights, biases


Nie mogłem tego zrobić na swój własny sposób, bo mi się kształty nie zgadzały.

In [11]:
def backward_propagation(x, y, a1, a2, output, weights, biases, learning_rate):
    m = y.shape[0]
    error = output - y

    # Obliczanie gradientow propagacji wstecznej dla wartwy wyjscia
    dW3 = (1 / m) * np.dot(a2.T, error)
    db3 = (1 / m) * np.sum(error, axis=0, keepdims=True)

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz2 = np.dot(error, weights[2].T) * (a2 * (1 - a2))
    dW2 = (1 / m) * np.dot(a1.T, dz2)
    db2 = (1 / m) * np.sum(dz2, axis=0, keepdims=True)

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz1 = np.dot(dz2, weights[1].T) * (a1 * (1 - a1))
    dW1 = (1 / m) * np.dot(x.T, dz1)
    db1 = (1 / m) * np.sum(dz1, axis=0, keepdims=True)

    # Aktualizacja wag i biasów
    weights[2] -= learning_rate * dW3
    biases[2] -= learning_rate * db3
    weights[1] -= learning_rate * dW2
    biases[1] -= learning_rate * db2
    weights[0] -= learning_rate * dW1
    biases[0] -= learning_rate * db1

    return weights, biases

Nie potrafiłem sobie poradzić z implementacją propagacji wstecznej, dlatego użyłem kodu znalezionego na necie.

# Test sieci neuronowej

In [12]:
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
y_predict = np.array([[0], [1], [1], [0]])  

learning_rate = 0.01
epochs = 5000

for epoch in range(epochs):
    # Propagacja w przód
    layer1, layer2, output = forward_propagation(x_train, weights, biases)

    # Obliczenie kosztu straty po dojściu na koniec sieci
    # loss = mse_loss(y_predict, output)
    loss = compute_cost(y_predict, output)

    # Propagacja wsteczna
    update_weights, update_biases = backward_propagation(x_train, y_predict, layer1, layer2, output, weights, biases, learning_rate)

    if epoch % 1000 == 0:
        print(f"Iteracje/Epoki {epoch}, Strata : {loss:.4f}")
        print(f"Wejscia: {x_train.flatten()}, Przewidywania sieci: {output.flatten()}, rzeczywiste wyniki {y_predict.flatten()}")


Iteracje/Epoki 0, Strata : 2.2506
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.0916287  0.15837234 0.09173521 0.1582806  0.09159354 0.15838986
 0.09170379 0.15829597], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 1000, Strata : 2.2157
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.11851698 0.13517148 0.11485456 0.13049782 0.11886771 0.13560314
 0.11536168 0.13112663], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 2000, Strata : 2.2130
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12374069 0.12482433 0.12354037 0.12461338 0.12534498 0.1265049
 0.12514114 0.1262902 ], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 3000, Strata : 2.2125
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12386221 0.12395425 0.12455668 0.124666   0.12542716 0.12555812
 0.12591611 0.12605946], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 4000, Strata : 2.2123
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12389971 0.12391144 0.12479755 0.12482522 0.12531729 0.12535408
 0.12592349 0.12597123]

Podsumowanie: wyniki są beznadziejne, dalekie od dobrych przewidywań. Czy jest to spowodowane zanikiem spadku gradientu? Bo funkcja straty ma bardzo niewielkie spadki podczas większych iteracji. Jak by modelowi ciężko było zaktualizować lepsze wagi. Pamiętam, że czytałem taką informacje iż funkcja sigmoid ma tedencje do problemu zanikających gradientów, ale czy to na pewno to?